# High-level RNN MXNet Example

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from common.params_lstm import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.3
MXNet:  0.12.1
GPU:  ['Tesla K80']


In [3]:
def create_symbol(CUDNN=True):
    # https://mxnet.incubator.apache.org/api/python/rnn.html
    data = mx.symbol.Variable('data')
    embedded_step = mx.symbol.Embedding(data=data, input_dim=MAXFEATURES, output_dim=EMBEDSIZE)
    
    # Fusing RNN layers across time step into one kernel
    # Improves speed but is less flexible
    # Currently only supported if using cuDNN on GPU
    if not CUDNN:
        gru_cell = mx.rnn.GRUCell(num_hidden=NUMHIDDEN)
    else:
        gru_cell = mx.rnn.FusedRNNCell(num_hidden=NUMHIDDEN, num_layers=1, mode='gru')
    
    begin_state = gru_cell.begin_state()
    # Call the cell to get the output of one time step for a batch.
    # TODO: TNC layout (sequence length, batch size, and feature dimensions) is faster for RNN
    outputs, states = gru_cell.unroll(length=MAXLEN, inputs=embedded_step, merge_outputs=False)
    
    fc1 = mx.symbol.FullyConnected(data=outputs[-1], num_hidden=2) 
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc1, label=input_y, name="softmax")
    return m

In [4]:
def init_model(m):
    if GPU:
        ctx = [mx.gpu(0)]
    else:
        ctx = mx.cpu()
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=[('data', (BATCHSIZE, MAXLEN))],
             label_shapes=[('softmax_label', (BATCHSIZE, ))])
    # Glorot-uniform initializer
    mod.init_params(initializer=mx.init.Xavier(rnd_type='uniform'))
    mod.init_optimizer(optimizer='Adam', 
                       optimizer_params=(('learning_rate', LR),
                                         ('beta1', BETA_1),
                                         ('beta2', BETA_2),
                                         ('epsilon', EPS)))
    return mod

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)

# Use custom iterator instead of mx.io.NDArrayIter() for consistency
# Wrap as DataBatch class
wrapper_db = lambda args: mx.io.DataBatch(data=[mx.nd.array(args[0])], label=[mx.nd.array(args[1])])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 5.48 s, sys: 313 ms, total: 5.8 s
Wall time: 5.8 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 81.3 ms, sys: 3.35 ms, total: 84.6 ms
Wall time: 84.1 ms


/anaconda/envs/py35/lib/python3.5/site-packages/mxnet/rnn/rnn_cell.py:675: UserWarning: NTC layout detected. Consider using TNC for FusedRNNCell for faster speed
  warnings.warn("NTC layout detected. Consider using "


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 789 ms, sys: 359 ms, total: 1.15 s
Wall time: 1.17 s


In [8]:
%%time
# 29s
# Train and log accuracy
metric = mx.metric.create('acc')
for j in range(EPOCHS):
    #train_iter.reset()
    metric.reset()
    #for batch in train_iter:
    for batch in map(wrapper_db, yield_mb(x_train, y_train, BATCHSIZE, shuffle=True)):
        model.forward(batch, is_train=True) 
        model.update_metric(metric, batch.label)
        model.backward()              
        model.update()
    print('Epoch %d, Training %s' % (j, metric.get()))

Epoch 0, Training ('accuracy', 0.77880608974358978)
Epoch 1, Training ('accuracy', 0.92219551282051282)
Epoch 2, Training ('accuracy', 0.96546474358974355)
CPU times: user 25.9 s, sys: 4.29 s, total: 30.2 s
Wall time: 28.8 s


In [9]:
%%time
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 2.57 s, sys: 350 ms, total: 2.92 s
Wall time: 2.7 s


In [10]:
print("Accuracy: ", sum(y_guess == y_test)/len(y_guess))

Accuracy:  0.85924
